In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [6]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [7]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.categorical_crossentropy(y_true, y_pred)
    return loss1 * 0.7 + loss2 * 0.3

In [8]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

W1007 23:50:20.700227 140175882671872 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 23:50:20.721671 140175882671872 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1007 23:50:20.731012 140175882671872 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1007 23:50:21.528933 140175882671872 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [10]:
model_path = 'result/mse_cce_models/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min')#, period=5)

In [12]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/500
2360/2360 [==============================] - 68s 29ms/step - loss: 36.6038

Epoch 00001: loss improved from 37.45407 to 36.60385, saving model to result/mse_cce_models/weights01-36.6038.h5
Epoch 2/500
2360/2360 [==============================] - 67s 29ms/step - loss: 36.5921

Epoch 00002: loss improved from 36.60385 to 36.59214, saving model to result/mse_cce_models/weights02-36.5921.h5
Epoch 3/500
2360/2360 [==============================] - 47s 20ms/step - loss: 36.5886

Epoch 00003: loss improved from 36.59214 to 36.58862, saving model to result/mse_cce_models/weights03-36.5886.h5
Epoch 4/500
2360/2360 [==============================] - 40s 17ms/step - loss: 36.5854

Epoch 00004: loss improved from 36.58862 to 36.58543, saving model to result/mse_cce_models/weights04-36.5854.h5
Epoch 5/500
2360/2360 [==============================] - 48s 20ms/step - loss: 36.5834

Epoch 00005: loss improved from 36.58543 to 36.58342, saving model to result/mse_cce_models/weights05-36.583

2360/2360 [==============================] - 36s 15ms/step - loss: 1.4465

Epoch 00047: loss improved from 1.47082 to 1.44650, saving model to result/mse_cce_models/weights47-1.4465.h5
Epoch 48/500
2360/2360 [==============================] - 36s 15ms/step - loss: 1.2941

Epoch 00048: loss improved from 1.44650 to 1.29413, saving model to result/mse_cce_models/weights48-1.2941.h5
Epoch 49/500
2360/2360 [==============================] - 35s 15ms/step - loss: 1.3044

Epoch 00049: loss did not improve from 1.29413
Epoch 50/500
2360/2360 [==============================] - 35s 15ms/step - loss: 1.2843

Epoch 00050: loss improved from 1.29413 to 1.28431, saving model to result/mse_cce_models/weights50-1.2843.h5
Epoch 51/500
2360/2360 [==============================] - 35s 15ms/step - loss: 1.1058

Epoch 00051: loss improved from 1.28431 to 1.10575, saving model to result/mse_cce_models/weights51-1.1058.h5
Epoch 52/500
2360/2360 [==============================] - 35s 15ms/step - loss: 1.2975

2360/2360 [==============================] - 35s 15ms/step - loss: 0.1287

Epoch 00153: loss did not improve from 0.12713
Epoch 154/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1444

Epoch 00154: loss did not improve from 0.12713
Epoch 155/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1317

Epoch 00155: loss did not improve from 0.12713
Epoch 156/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1278

Epoch 00156: loss did not improve from 0.12713
Epoch 157/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1417

Epoch 00157: loss did not improve from 0.12713
Epoch 158/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1471

Epoch 00158: loss did not improve from 0.12713
Epoch 159/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1291

Epoch 00159: loss did not improve from 0.12713
Epoch 160/500
2360/2360 [==============================] - 35s

2360/2360 [==============================] - 35s 15ms/step - loss: 0.0671

Epoch 00267: loss did not improve from 0.06483
Epoch 268/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0694

Epoch 00268: loss did not improve from 0.06483
Epoch 269/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0729

Epoch 00269: loss did not improve from 0.06483
Epoch 270/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0654

Epoch 00270: loss did not improve from 0.06483
Epoch 271/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0640

Epoch 00271: loss improved from 0.06483 to 0.06402, saving model to result/mse_cce_models/weights271-0.0640.h5
Epoch 272/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0659

Epoch 00272: loss did not improve from 0.06402
Epoch 273/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0848

Epoch 00273: loss did not improve from 0.0640

2360/2360 [==============================] - 36s 15ms/step - loss: 0.0502

Epoch 00381: loss did not improve from 0.03981
Epoch 382/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0500

Epoch 00382: loss did not improve from 0.03981
Epoch 383/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0510

Epoch 00383: loss did not improve from 0.03981
Epoch 384/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0505

Epoch 00384: loss did not improve from 0.03981
Epoch 385/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0479

Epoch 00385: loss did not improve from 0.03981
Epoch 386/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0569

Epoch 00386: loss did not improve from 0.03981
Epoch 387/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0427

Epoch 00387: loss did not improve from 0.03981
Epoch 388/500
2360/2360 [==============================] - 35s

2360/2360 [==============================] - 36s 15ms/step - loss: 0.0463

Epoch 00499: loss did not improve from 0.03360
Epoch 500/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0374

Epoch 00500: loss did not improve from 0.03360


In [13]:
model_json = lstm_autoencoder.to_json()
filename = 'last_mse_lstmae_cce'  #input('filename: ') #
with open('result/mse_cce_models/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('result/mse_cce_models/weights_' +  filename + '.h5')

In [7]:
filename = "last_mse_lstmae_cce"
loaded_model = model_from_json(open('result/mse_cce_models/' +filename + '.json').read())
loaded_model.load_weights('result/mse_cce_models/weights_' + filename + '.h5')

W1010 15:50:00.245121 140290105825024 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 15:50:00.273124 140290105825024 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1010 15:50:00.276480 140290105825024 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1010 15:50:00.816859 140290105825024 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [14]:
loaded_model = lstm_autoencoder

In [8]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

0.6315540421589082


In [ ]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])